# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [126]:
c.execute("""select *
                    from customers
                    order by creditLimit desc;""")


df = pd.DataFrame(c.fetchall())
show(df).head(10)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,334,Suominen Souveniers,Suominen,Kalle,+358 9 8045 555,Software Engineering Center,SEC Oy,Espoo,,FIN-02271,Finland,1501,98800.00
1,166,Handji Gifts& Co,Victorino,Wendy,+65 224 1555,106 Linden Road Sandown,2nd Floor,Singapore,,069045,Singapore,1612,97900.00
2,167,Herkku Gifts,Oeztan,Veysel,+47 2267 3215,Brehmen St. 121,PR 334 Sentrum,Bergen,,N 5804,Norway,1504,96800.00
3,186,"Toys of Finland, Co.",Karttunen,Matti,90-224 8555,Keskuskatu 45,,Helsinki,,21240,Finland,1501,96500.00
4,455,Super Scale Inc.,Murphy,Leslie,2035559545,567 North Pendale Street,,New Haven,CT,97823,USA,1286,95400.00
5,299,"Norway Gifts By Mail, Co.",Klaeboe,Jan,+47 2212 1555,Drammensveien 126A,PB 211 Sentrum,Oslo,,N 0106,Norway,1504,95100.00
6,406,Auto Canal+ Petit,Perrier,Dominique,(1) 47.55.6555,"25, rue Lauriston",,Paris,,75016,France,1337,95000.00
7,320,Mini Creations Ltd.,Huang,Wing,5085559555,4575 Hillside Dr.,,New Bedford,MA,50553,USA,1188,94500.00
8,398,"Tokyo Collectables, Ltd",Shimamura,Akiko,+81 3 3584 0555,2-2-8 Roppongi,,Minato-ku,Tokyo,106-0032,Japan,1621,94400.00
9,240,giftsbymail.co.uk,Bennett,Helen,(198) 555-8888,Garden House,Crowther Way 23,Cowes,Isle of Wight,PO31 7PJ,UK,1501,93900.00


In [142]:
c.execute("""select creditLimit, firstName, lastName 
                    from customers c
                    join employees e
                    on c.salesRepEmployeeNumber = e.employeeNumber
                    where lastName = 'Firrelli'and
                    creditLimit > 15000;""")
df = pd.DataFrame(c.fetchall())
show(df)

,creditLimit,firstName,lastName
0,43400.00,Julie,Firrelli
1,68700.00,Julie,Firrelli
2,70700.00,Julie,Firrelli
3,81100.00,Julie,Firrelli
4,85100.00,Julie,Firrelli
5,94500.00,Julie,Firrelli


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [147]:
c.execute("""select employeeNumber, firstName, lastName, 
                    count(creditLimit) as no_over_15k
                    from employees e
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    where creditLimit >= 15000
                    group by 1, 2, 3
                    ;""")
df = pd.DataFrame(c.fetchall())
show(df)

,employeeNumber,firstName,lastName,no_over_15k
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


In [2]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd 

conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

In [11]:
def show(database):
    database.columns = [x[0] for x in c.description]
    return database

## Select the Average Number of Orders on a Product by Product Basis

In [78]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select AVG(numOrders)
                    from (select productCode,
                                 count(distinct orderNumber) as numOrders
                                 from products
                                 join orderdetails
                                 using(productCode)
                                 group by 1
                         );
          """
         )
df = pd.DataFrame(c.fetchall())
show(df)

,AVG(numOrders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [33]:
c.execute("""select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';""")
df = pd.DataFrame(c.fetchall())
show(df)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [29]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""select customerNumber, contactLastName, contactFirstName
                    from customers
                    where customerNumber in (select customerNumber from orders where orderDate = '2003-01-31')""")
df = pd.DataFrame(c.fetchall())
show(df)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [59]:
#Your code here
c.execute("""select productName, count(distinct orderNumber) as no_orders, sum(quantityOrdered) as units_sold
                    from products
                    join orderdetails
                    using(productCode)
                    group by 1
                    order by count(distinct orderNumber) desc""")
df = pd.DataFrame(c.fetchall())
show(df).head()

,productName,no_orders,units_sold
0,1992 Ferrari 360 Spider red,53,1808
1,18th Century Vintage Horse Carriage,28,907
2,1900s Vintage Bi-Plane,28,940
3,1900s Vintage Tri-Plane,28,1009
4,1913 Ford Model T Speedster,28,1038


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [66]:
#Your code here
c.execute("""select productName, count(distinct customerNumber) as no_ppl_ordered
                    from products p
                    join orderdetails od
                    using(productCode)
                    join orders o
                    using(orderNumber)
                    group by 1
                    order by no_ppl_ordered desc""")
df = pd.DataFrame(c.fetchall())
show(df).head()

,productName,no_ppl_ordered
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [114]:
c.execute("""select distinct employeeNumber, firstName, lastName, 
                    officeCode, o.city
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders ord
                    using(customerNumber)
                    join orderdetails
                    using(orderNumber)
                    where productCode in (select productCode
                                                    from products
                                                    join orderdetails
                                                    using(productCode)
                                                    join orders
                                                    using(orderNumber)
                                                    group by 1
                                                    having count(distinct customerNumber) < '20'
                                                    );""")
                    
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,employeeNumber,firstName,lastName,officeCode,city
0,1216,Steve,Patterson,2,Boston
1,1504,Barry,Jones,7,London
2,1286,Foon Yue,Tseng,3,NYC
3,1370,Gerard,Hernandez,4,Paris
4,1401,Pamela,Castillo,4,Paris
5,1323,George,Vanauf,3,NYC
6,1621,Mami,Nishi,5,Tokyo
7,1501,Larry,Bott,7,London
8,1165,Leslie,Jennings,1,San Francisco
9,1337,Loui,Bondur,4,Paris


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!